In [1]:
import torch 
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

# 디버깅 모듈 설치
import pdb

# 우리가 사용할 컴퓨터를 check하는 부분, cpu/gpu 지원받을지. cuda라고 나오면 엔비디아의 gpu 서버임
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [2]:
# Hyper Parameter 지정
input_size = 784 # 입력값이  28 x 28 x 1 / 2차원 이미지를 1차원으로 주욱 펼친 한 장의 이미지 값
hidden_size = 500 # hidden layer의 unit 수 / 784를 다시 500으로 줄이고
num_classes = 10 # 카테고리 개수, 딥러닝 결과가 10개 나온다.손글씨의 분류는 10개 / 최종적으로는 10개가 나와야 한다. 그림
num_epochs = 5
batch_size = 100 # 이미지가 아니고 크기가 크지 않은 모델에서는 굳이 쓰일 필요가 없다.
learning_rate = 0.001

# 데이터셋 로딩 -- 2번에 걸쳐서 로딩
# 1단계 - 물리적인 위치에 데이터 저장. 다운로드(training / test)
# train_dataset
train_dataset = torchvision.datasets.CIFAR10(root='../../data', 
                                           train=True,
                                           transform=transforms.ToTensor(),
                                           download=True)
# test_dataset
test_dataset = torchvision.datasets.CIFAR10(root='../../data', 
                                           train=False,
                                           transform=transforms.ToTensor())

# 2단계 - 네트워크에 100개씩 잘라서 로드함
# 2 step. DataLoader. BatchSize를 이용함.
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size = batch_size,
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                           batch_size = batch_size,
                                           shuffle=False)

print('1',train_dataset)
print('2',test_dataset)
print('3',train_loader)


Extracting ../../data/cifar-10-python.tar.gz to ../../data
1 Dataset CIFAR10
    Number of datapoints: 50000
    Root location: ../../data
    Split: Train
    StandardTransform
Transform: ToTensor()
2 Dataset CIFAR10
    Number of datapoints: 10000
    Root location: ../../data
    Split: Test
    StandardTransform
Transform: ToTensor()
3 <torch.utils.data.dataloader.DataLoader object at 0x7f945e60bfd0>


In [30]:
class ConvNet(nn.Module):
  def __init__(self, num_classes = 10):
      """CNN Builder."""
      super(ConvNet, self).__init__()

      self.conv_layer = nn.Sequential(

          # Conv Layer block 1
          nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1),
          nn.BatchNorm2d(32),
          nn.ReLU(inplace=True),
          nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
          nn.ReLU(inplace=True),
          nn.MaxPool2d(kernel_size=2, stride=2),

          # Conv Layer block 2
          nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
          nn.BatchNorm2d(128),
          nn.ReLU(inplace=True),
          nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1),
          nn.ReLU(inplace=True),
          nn.MaxPool2d(kernel_size=2, stride=2),
          nn.Dropout2d(p=0.05),

          # Conv Layer block 3
          nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1),
          nn.BatchNorm2d(256),
          nn.ReLU(inplace=True),
          nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
          nn.ReLU(inplace=True),
          nn.MaxPool2d(kernel_size=2, stride=2),
      )


      self.fc_layer = nn.Sequential(
          nn.Dropout(p=0.1),
          nn.Linear(4096, 1024),
          nn.ReLU(inplace=True),
          nn.Linear(1024, 512),
          nn.ReLU(inplace=True),
          nn.Dropout(p=0.1),
          nn.Linear(512, 10)
      )


  def forward(self, x):
      """Perform forward."""
      
      # conv layers
      x = self.conv_layer(x)
      
      # flatten
      x = x.view(x.size(0), -1)
      
      # fc layer
      x = self.fc_layer(x)

      return x

In [31]:
model = ConvNet(num_classes).to(device)

In [32]:
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [33]:
total_step = len(train_loader)
for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_loader):
    images = images.to(device)
    labels = labels.to(device)

    # 디버깅 부분
    # pdb.set_trace() # 1. l  2. size 확인

    outputs = model(images) # 누구나 이 부분 디버깅을 해봐야 함.
    # 여기에서 images.shape를 찍으면 무슨 값이 나올까?
    
     # 디버깅 부분
    # pdb.set_trace() # 3.


    loss = loss_function(outputs, labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if(i+1) % 100 ==0:
            print('Epoch[{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, total_step, loss.item()))


Epoch[1/5], Step [100/500], Loss: 1.7211
Epoch[1/5], Step [200/500], Loss: 1.4089
Epoch[1/5], Step [300/500], Loss: 1.4796
Epoch[1/5], Step [400/500], Loss: 1.0347
Epoch[1/5], Step [500/500], Loss: 1.1979
Epoch[2/5], Step [100/500], Loss: 1.1282
Epoch[2/5], Step [200/500], Loss: 0.9125
Epoch[2/5], Step [300/500], Loss: 0.9248
Epoch[2/5], Step [400/500], Loss: 0.6783
Epoch[2/5], Step [500/500], Loss: 0.7643
Epoch[3/5], Step [100/500], Loss: 0.6297
Epoch[3/5], Step [200/500], Loss: 0.8209
Epoch[3/5], Step [300/500], Loss: 0.7727
Epoch[3/5], Step [400/500], Loss: 0.7273
Epoch[3/5], Step [500/500], Loss: 0.7391
Epoch[4/5], Step [100/500], Loss: 0.4863
Epoch[4/5], Step [200/500], Loss: 0.7013
Epoch[4/5], Step [300/500], Loss: 0.5652
Epoch[4/5], Step [400/500], Loss: 0.6588
Epoch[4/5], Step [500/500], Loss: 0.5478
Epoch[5/5], Step [100/500], Loss: 0.5171
Epoch[5/5], Step [200/500], Loss: 0.6098
Epoch[5/5], Step [300/500], Loss: 0.6269
Epoch[5/5], Step [400/500], Loss: 0.4330
Epoch[5/5], Step

In [29]:
model.eval()
with torch.no_grad(): # 실제로 학습할 필요가 없으면 no.grad()구문을 넣어준다.
    correct = 0
    total = 0
    for images, labels in test_loader: # 인덱스를 굳이 안받아도 되니깐 enumeratorr가 빠졌다.
        images = images.to(device) #한줄로 죽 펼친 다음에 입력받아서
        labels = labels.to(device) # 라벨도 마찬가지로
        outputs = model(images) #모델에 넣고
        _, predicted = torch.max(outputs.data, 1)
        # total은 푼 문제, correct는 맞춘 문제수..이런식으로 카운팅이 들어간다
     
        total += labels.size(0) 
        correct += (predicted == labels).sum().item()
        print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

Test Accuracy of the model on the 10000 test images: 71.0 %
Test Accuracy of the model on the 10000 test images: 67.0 %
Test Accuracy of the model on the 10000 test images: 64.66666666666667 %
Test Accuracy of the model on the 10000 test images: 64.75 %
Test Accuracy of the model on the 10000 test images: 65.0 %
Test Accuracy of the model on the 10000 test images: 65.66666666666667 %
Test Accuracy of the model on the 10000 test images: 66.14285714285714 %
Test Accuracy of the model on the 10000 test images: 65.0 %
Test Accuracy of the model on the 10000 test images: 65.55555555555556 %
Test Accuracy of the model on the 10000 test images: 65.6 %
Test Accuracy of the model on the 10000 test images: 66.0909090909091 %
Test Accuracy of the model on the 10000 test images: 66.66666666666667 %
Test Accuracy of the model on the 10000 test images: 66.53846153846153 %
Test Accuracy of the model on the 10000 test images: 66.14285714285714 %
Test Accuracy of the model on the 10000 test images: 66.